# Example notebook of a Ancestry QC analysis

The present notebook serves as a guide of how use the `IDEAL-GENOM-QC` library to perform a ancestry quality control. We intend to show a possible use, because each user can adapt it to its particular needs. Up to this moment the library can only detects outliers from an homogenous population that overlaps with one of the `SuperPop` present in the **1000 Genomes** data.

In this notebook the procedure to perform the ancestty quality control is more detailed so the user can get a deeper understanding of all the steps executed in this part of the pipeline.

Let us import the required libraries.

In [ ]:
import sys
import os

from pathlib import Path

# add parent directory to path
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

library_path = Path(library_path)

from ideal_genom_qc.AncestryQC import AncestryQC

In the next cell the path variables associated with the project are set.

Moreover, since each user can have a slightly different choices for the LD regions, the user can provide its own file. Nevertheless, we provide the functionality of automatically fetching high LD regions for builts **GRCh37** and **GRCH38**. 

Also, the user can provide the path to the reference genome files or let the library fetch the data automatically.

When giving the path to the input files, the user should take into account that the input files of the ancestry check must the output of the sample QC.

In [ ]:
input_path = Path('/path/to/input/data/')
input_name = 'input-prefix'
output_path= Path('/path/to/output/data/')
output_name= 'output-prefix'
high_ld_file = Path('path/to/ld-regions/file') # if not available, set to Path()

In the next cell we define a dictionary with the parameters to execute the ancestry QC.

The explanation of the parameters is the following:

1. `ind_pair`: parameter of **PLINK1.9** `--indep-pairwise`.
2. `pca`: number of principal components to be computed, parameter `--pca` from **PLINK1.9**.
3. `maf`: minor allele frequency, parameter `--maf` of **PLINK1.9**.
4. `ref_threshold`: number of standard deviations from the mean of the reference panel `SuperPop` to a sample be considered a possible outlier.
5. `stu_threshold`: number of standard deviations from the mean of the study population to flag a sample as possible outlier.
6. `reference_pop`: Super population from the reference panel considered as reference for the study.
7. `num_pcs`: number of principal components used to flag a sample as outlier.

In [ ]:
ancestry_params ={
    "ind_pair"     : [50, 5, 0.2],
    "pca"          : 10,
    "maf"          : 0.01,
    "ref_threshold": 4,
    "stu_threshold": 4,
    "reference_pop": "SAS",
    "num_pcs"      : 10,
}

In [ ]:
ancestry_qc = AncestryQC(
    input_path = input_path, 
    input_name = input_name, 
    output_path= output_path, 
    output_name= output_name, 
    high_ld_file= high_ld_file,
    recompute_merge=True, # if True, it will recompute the merge of the input files
    built='38', # '38' it is the default value 
)

In [ ]:
ancestry_qc_steps = {
    'merge_study_reference'    : (ancestry_qc.merge_reference_study, {"ind_pair":ancestry_params['ind_pair']}),
    #'delete_intermediate_files': (ancestry_qc._clean_merging_dir, {}),
    'pca_analysis'             : (ancestry_qc.run_pca, 
        {
            "ref_population": ancestry_params['reference_pop'],
            "pca":ancestry_params['pca'],
            "maf":ancestry_params['maf'],
            "num_pca":ancestry_params['num_pcs'],
            "ref_threshold":ancestry_params['ref_threshold'],
            "stu_threshold":ancestry_params['stu_threshold'],
        }
    ),
}

step_description = {
    'merge_study_reference'    : "Merge reference genome with study genome",
    #'delete_intermediate_files': "Delete intermediate files generated during merging",
    'pca_analysis'             : "Run a PCA analysis to perfom ancestry QC"
}

for name, (func, params) in ancestry_qc_steps.items():
    print(f"\033[1m{step_description[name]}.\033[0m")
    func(**params)

In [2]:
import pandas as pd

df_psam = pd.read_csv(
    '/home/luis/CGE/ideal-genom-qc/data/1000genomes_built_38/1kG_phase3_GRCh38.psam',
    sep='\t',
    header=0,
)
df_psam = df_psam[['#IID', 'SuperPop']].copy()

In [13]:
df_psam['FID'] = '0'
df_psam = df_psam[['FID', '#IID', 'SuperPop']].copy()
df_psam.to_csv(
    '/media/luis/LaCie/data1/qc1000genomes/dependables/population_info.csv',
    sep='\t',
    index=False,
    header=True
)

In [ ]:
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Create a new column with random values 1 and 2
df_psam['pheno'] = np.random.choice([1, 2], size=len(df_psam))

# Display the first few rows to verify
df_psam.head()

,FID,#IID,SuperPop,pheno
0,0,HG00096,EUR,1
1,0,HG00097,EUR,2
2,0,HG00099,EUR,1
3,0,HG00100,EUR,1
4,0,HG00101,EUR,1


In [16]:
df_psam[['FID', "#IID", 'pheno']].to_csv(
    '/media/luis/LaCie/data1/qc1000genomes/dependables/pheno_info.csv',
    sep='\t',
    index=False,
    header=False
)

In [4]:
df_bim = pd.read_csv(
    '/media/luis/LaCie/rawdata-sexupdated/outputData/sample_qc_results/clean_files/qc-results-clean-samples.bim',
    sep='\t',
    header=None,
    names=['chr', 'snp', 'cm', 'pos', 'a1', 'a2']
)
df_bim

,chr,snp,cm,pos,a1,a2
0,1,1:49554:A:G,0,49554,G,A
1,1,1:115746:C:T,0,115746,T,C
2,1,1:598858:A:C,0,598858,C,A
3,1,1:629488:C:T,0,629488,T,C
4,1,1:630053:C:T,0,630053,T,C
...,...,...,...,...,...,...
1842631,24,Y:26671311:C:A,0,26671311,A,C
1842632,24,Y:26671347:A:T,0,26671347,T,A
1842633,24,Y:26671432:G:A,0,26671432,A,G
1842634,24,Y:26671489:A:G,0,26671489,G,A


In [ ]:
# Define the path to the BIM file
large_bim_path = '/media/luis/LaCie/data1/qc1000genomes/inputData/all_hg38-filtered-renamed.bim'

# Define column names for BIM file
bim_columns = ['chr', 'snp', 'cm', 'pos', 'a1', 'a2']

# Initialize an empty list to store chunks
chunks = []

# Load the file in chunks of 500,000 rows
chunk_size = 500000
for chunk in pd.read_csv(large_bim_path, sep='\t', header=None, names=bim_columns, 
                         chunksize=chunk_size):
    # Process each chunk (e.g., print its shape)
    print(f"Loaded chunk with shape: {chunk.shape}")
    
    # Optionally store the chunk for further processing
    chunks.append(chunk)

# If you need the entire dataframe after processing in chunks
# Be careful with this if the file is very large
# df_large_bim = pd.concat(chunks)
# print(f"Total shape of combined data: {df_large_bim.shape}")

# Alternatively, print summary statistics without loading the entire file
print("\nSample from first chunk:")
print(chunks[0].head())

# Count total number of rows processed
total_rows = sum(len(chunk) for chunk in chunks)
print(f"\nTotal number of rows processed: {total_rows}")


KeyboardInterrupt: 